In [ ]:
# Install all required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install peft
!pip install datasets
!pip install accelerate
!pip install tqdm
!pip install tensorboard
!pip install bitsandbytes
!pip install sentencepiece
!pip install protobuf

# Verify installations
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.3 MB/s eta 0:00:00
PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA version: 12.6
Number of GPUs: 1


In [ ]:
# GPU VERSION - Remove TPU imports, add GPU imports
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import json

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Load model and tokenizer
model_name = "tiiuae/Falcon3-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,  # Use float16 for GPU memory efficiency
    device_map="auto",  # Automatically handle GPU placement
    trust_remote_code=True
)

# Add padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


Using device: cuda
GPU: Tesla T4
GPU Memory: 15.83 GB


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("json", data_files="./data/hf_train.jsonl.jsonl")["train"]
eval_dataset = load_dataset("json", data_files="./data/hf_eval.jsonl")["train"]

print("Train samples:", len(train_dataset))
print("Eval samples:", len(eval_dataset))



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Train samples: 8191
Eval samples: 911


In [ ]:
import json
from transformers import AutoTokenizer
from tqdm import tqdm

# === Config ===
file_path = "./data/hf_train.jsonl"

total_tokens = 0

total_tokens = 0
num_examples = 0

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)

        # Combine instruction + output as training text
        text = obj.get("instruction", "") + "\n" + str(obj.get("output", ""))

        # Tokenize without truncation
        tokens = tokenizer(text, truncation=False, return_tensors=None)
        total_tokens += len(tokens["input_ids"])
        num_examples += 1

print(f"📊 Total tokens in dataset: {total_tokens:,}")
print(f"📈 Average tokens per example: {total_tokens/num_examples:.2f}")
print(f"📦 Number of examples: {num_examples}")


📊 Total tokens in dataset: 6,095,024
📈 Average tokens per example: 744.11
📦 Number of examples: 8191


In [ ]:
print(train_dataset[7017]['instruction'])

#User
I am working on a project to find the relevance of news articles to different categories. I need to be able to quickly classify news articles into categories such as politics, sports, and technology.
<Reference API>:{"domain": "Natural Language Processing Zero-Shot Classification", "framework": "Transformers", "functionality": "Zero-Shot Classification", "api_name": "cross-encoder/nli-deberta-v3-xsmall", "api_call": "pipeline('zero-shot-classification', model='cross-encoder/nli-deberta-v3-xsmall')", "api_arguments": ["sent", "candidate_labels"], "python_environment_requirements": ["sentence_transformers", "transformers"], "example_code": "from transformers import pipeline\nclassifier = pipeline('zero-shot-classification', model='cross-encoder/nli-deberta-v3-xsmall')\nsent = 'Apple just announced the newest iPhone X'\ncandidate_labels = ['technology', 'sports', 'politics']\nres = classifier(sent, candidate_labels)\nprint(res)", "performance": {"dataset": {"SNLI-test": "91.64", "MN

In [ ]:
def preprocess(examples):
    texts = []
    for instr, out in zip(examples["instruction"], examples["output"]):
        prompt = f"{instr}\n"

        # Ensure output is a string
        if isinstance(out, list):
            output_text = " ".join(out)
        else:
            output_text = str(out)

        texts.append(prompt + output_text)

    return tokenizer(texts, truncation=False, padding=False,)

train_dataset = train_dataset.map(preprocess, batched=True, remove_columns=["instruction","output"])
eval_dataset = eval_dataset.map(preprocess, batched=True, remove_columns=["instruction","output"])


Map:   0%|          | 0/8191 [00:00<?, ? examples/s]

Map:   0%|          | 0/911 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel


# Load previous LoRA adapter weights on top
model1 = PeftModel.from_pretrained(model, "./falcon3b_instruct_1stepoch", device_map="auto")

# Continue training as usual
model1.train()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(131072, 3072)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
model1.enable_input_require_grads()


# Enable gradients for PEFT parameters
for name, param in model1.named_parameters():
    if 'lora' in name.lower():
        param.requires_grad = True
        print(f"Enabled gradients for: {name}")

Enabled gradients for: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
Enabled gradients for: base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
Enabled gradients for: base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
Enabled gradients for: base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
Enabled gradients for: base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
Enabled gradients for: base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
Enabled gradients for: base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
Enabled gradients for: base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
Enabled gradients for: base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight
Enabled gradients for: base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight
Enabled gradients for: base_model.model.model.layers.1.self_

In [ ]:
# print total and a handful of trainable param names
trainable = [(n, p.numel()) for n,p in model1.named_parameters() if p.requires_grad]
print("num trainable params:", sum(p for _,p in trainable))
print("sample trainable names:", [n for n,_ in trainable][:20])


num trainable params: 7208960
sample trainable names: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
 from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# ================== TrainingArguments ==================
training_args = TrainingArguments(
    output_dir="./falcon3b_instruct_lora_2nd",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="steps",
    save_steps=200,
    eval_strategy="steps",
    eval_steps=80,                     # Evaluate more frequently (was 100)
    learning_rate=7e-5,                # Slightly lower LR for 2nd epoch
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    fp16=True,
    gradient_checkpointing=True,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    remove_unused_columns=False,
    push_to_hub=False,
    save_total_limit=3,
    warmup_ratio=0.03,                       # Add warmup for stability
    weight_decay=0.01,                       # Add slight weight decay
)



# ================== Data Collator ==================
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# ================== Trainer ==================
trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
)

# ================== GPU Logging (replaces TPU master_print) ==================
print("Trainer initialized ✅")
print(f"Train samples: {len(train_dataset)}, Eval samples: {len(eval_dataset)}")

# ================== GPU Memory Info (replaces TPU memory info) ==================
if torch.cuda.is_available():
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"GPU memory cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
else:
    print("No GPU available - using CPU")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Trainer initialized ✅
Train samples: 8191, Eval samples: 911
GPU memory allocated: 6.48 GB
GPU memory cached: 7.32 GB


In [ ]:

model1.print_trainable_parameters()


trainable params: 7,208,960 || all params: 3,234,864,128 || trainable%: 0.2229


In [ ]:
# GPU Training (replaces TPU training)
print("🚀 Starting training...")

# Print GPU memory before training (replaces TPU memory)
if torch.cuda.is_available():
    print(f"[BEFORE TRAIN] GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"[BEFORE TRAIN] GPU memory cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
else:
    print("No GPU memory to check")

# Run training
train_result = trainer.train()

# Print GPU memory after training (replaces TPU memory)
if torch.cuda.is_available():
    print(f"[AFTER TRAIN] GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"[AFTER TRAIN] GPU memory cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
else:
    print("No GPU memory to check")

# Save model
trainer.save_model("./falcon3b_instruct_2ndepoch")
print("✅ Training finished and model saved!")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 2023}.


🚀 Starting training...
[BEFORE TRAIN] GPU memory allocated: 6.48 GB
[BEFORE TRAIN] GPU memory cached: 7.32 GB


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
80,0.554000,0.589848
160,0.547600,0.576334


Step,Training Loss,Validation Loss
80,0.554000,0.589848
160,0.547600,0.576334
240,0.542100,0.564645
320,0.518300,0.555295
400,0.533200,0.548919
480,0.535900,0.545196


[AFTER TRAIN] GPU memory allocated: 6.56 GB
[AFTER TRAIN] GPU memory cached: 11.88 GB
✅ Training finished and model saved!


In [ ]:
from google.colab import files
import os

folder_name = "./falcon3b_instruct_2ndepoch"
zip_name = folder_name + ".zip"

# Create zip file
!zip -r {zip_name} {folder_name}

# Download the zip file
files.download(zip_name)
